In [1]:
import time
start = time.time()

In [2]:
! nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-16ac6ff8-69b2-ff13-4397-cb2794269e47)
GPU 1: Tesla T4 (UUID: GPU-b48902e0-8b42-3cdb-3d79-227acf0c8cc6)


In [3]:
%%time

! pip install -qq -U langchain tiktoken pypdf chromadb faiss-gpu
! pip install -qq -U transformers InstructorEmbedding sentence_transformers
! pip install -qq -U bitsandbytes bitsandbytes-cuda117

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
cmudict 1.0.13 requires importlib-metadata<6.0.0,>=5.1.0, but you have importlib-metadata 6.0.1 which is incompatible.
dask-cuda 23.8.0 requires dask==2023.7.1, but you have dask 2023.9.0 which is incompatible.
dask-cuda 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.2 which is incompatible.
dask-cudf 23.8.0 requires dask==2023.7.1, but you have dask 2023.9.0 which is incompatible.
dask-cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.2 which is incompatible.
distributed 2023.7.1 requires dask==2023.7.1, but yo

In [4]:
import warnings
warnings.filterwarnings("ignore")

import os
import textwrap
import shutil
import pandas as pd
import glob

import langchain
from langchain.llms import HuggingFacePipeline

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline

#quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)

#os.environ['TRANSFORMERS_CACHE'] = 'cache/'

print(langchain.__version__)

0.0.335


In [5]:
### Multi-document retriever
from langchain.vectorstores import Chroma, FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA, VectorDBQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [6]:
%%time
print('\nDownloading model: ', '\n\n')
tokenizer = AutoTokenizer.from_pretrained("DAMO-NLP-MT/polylm-1.7b", use_fast=True)
model = AutoModelForCausalLM.from_pretrained("DAMO-NLP-MT/polylm-1.7b",
                                             load_in_8bit=True,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             low_cpu_mem_usage=True,
                                             trust_remote_code=True,
                                             #cache_dir="cache/",
                                             #offload_folder="save_folder"
#                                            rope_scaling = {'type': 'dynamic', 'factor': 2.0}
                                            )
max_len = 1024 # 512
task = "text-generation"
T = 0.1

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


CPU times: user 15.7 s, sys: 14.1 s, total: 29.8 s
Wall time: 44.4 s


In [7]:
pipe = pipeline(
    task=task,
    model=model, 
    tokenizer=tokenizer, 
    max_length=max_len,
    temperature=T,
    top_p=0.95,
    repetition_penalty=1.15,
    max_new_tokens=1024
)

llm = HuggingFacePipeline(pipeline=pipe)

In [8]:
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7f30d7f6b850>)

In [9]:
shutil.copytree('/kaggle/input/mkvc-9-finalmix/kaggle/working/mk-vectordb-chroma-nine', '/kaggle/working/mk-vectordb-chroma')

'/kaggle/working/mk-vectordb-chroma'

In [10]:
%%time

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="LazarusNLP/simcse-indobert-base", model_kwargs={"device": "cuda"})

vectordb = Chroma(persist_directory='mk-vectordb-chroma',embedding_function=instructor_embeddings,)
#vectordb.persist()

load INSTRUCTOR_Transformer
max_seq_length  512
CPU times: user 2.52 s, sys: 1.01 s, total: 3.53 s
Wall time: 19.7 s


In [11]:
%%time
retriever = vectordb.as_retriever(search_kwargs={"k": 3, "search_type" : "similarity"})
#retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 6, "lambda_mult": 0.25})

qa_chain = RetrievalQA.from_chain_type(llm=llm, 
                                       chain_type="stuff", 
                                       retriever=retriever, 
                                       return_source_documents=True,
                                       verbose=False)

CPU times: user 684 µs, sys: 0 ns, total: 684 µs
Wall time: 697 µs


In [12]:
qa_chain

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7f30d7f6b850>)), document_variable_name='context'), return_source_documents=True, retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceInstructEmbeddings'], vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x7f30d7f6be50>, search_kwargs={'k': 3, 'search_type': 'similarity'}))

In [13]:
def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [14]:
def llm_ans(query):
    llm_response = qa_chain(query)
    ans = process_llm_response(llm_response)
    return ans

In [15]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(256000, 2048)
    (wpe): Embedding(2048, 2048)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Linear8bitLt(in_features=2048, out_features=6144, bias=True)
          (c_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Linear8bitLt(in_features=2048, out_features=8192, bias=True)
          (c_proj): Linear8bitLt(in_features=8192, out_features=2048, bias=True)
          (act): FastGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e-0

In [16]:
dataset = pd.read_csv('/kaggle/input/mkri-qa-3/dataset.csv',sep=';')
dataset

,no,question,a,b,c,d,e,answer,explained,source,qa_comb
0,1,Berdasarkan Pasal 24C ayat (1) Undang-Undang D...,Mahkamah Agung berwenang mengadili pada tingka...,Mahkamah Konstitusi berwenang mengadili pada t...,Kepolisian Negara Indonesia mengadili pada tin...,Presiden Republik Indonesia mengadili pada tin...,Tentara Negara Indonesia mengadili pada tingka...,b,Mahkamah Konstitusi berwenang mengadili pada t...,3282_putusan_mkri_6516.pdf,Berdasarkan Pasal 24C ayat (1) Undang-Undang D...
1,2,Permohonan diajukan dalam jangka waktu paling ...,"Pasal 70 ayat (3) UU MK, Pasal 111 ayat (2) UU...","Pasal 74 ayat (3) UU MK, Pasal 474 ayat (2) UU...","Pasal 74 ayat (3) UU MK, Pasal 474 ayat (2) UU...","Pasal 74 ayat (3) UU MK, Pasal 474 ayat (2) UU...","Pasal 74 ayat (3) UU MK, Pasal 474 ayat (2) UU...",d,"Bahwa berdasarkan Pasal 74 ayat (3) UU MK, Pas...",3270_putusan_mkri_6508.pdf,Permohonan diajukan dalam jangka waktu paling ...
2,3,Pasal 1 ayat (3) UUD 1945 yang berbunyi…,Negara Indonesia adalah negara Hukum,Negara Indonesia adalah negara Islam,Negara Indonesia adalah negara liberal,Negara Indonesia adalah negara kepulauan,Negara Indonesia adalah negara Kristen,a,"Pasal 1 ayat (3) UUD 1945 yang berbunyi ""Negar...",1338_putusan_sidang_114%20PUU%202012%20-%20KUH...,Pasal 1 ayat (3) UUD 1945 yang berbunyi…\n A. ...
3,4,Bahwa dengan demikian norma Pasal 244 KUHAP te...,Pasal 21D ayat (1) UUD 1945,Pasal 22D ayat (1) UUD 1945,Pasal 23D ayat (1) UUD 1945,Pasal 24D ayat (1) UUD 1945,Pasal 28D ayat (1) UUD 1945,e,Bahwa dengan demikian norma Pasal 244 KUHAP te...,1338_putusan_sidang_114%2520PUU%25202012%2520-...,Bahwa dengan demikian norma Pasal 244 KUHAP te...
4,5,Bunyi rumusan Pasal 244 KUHAP...,Keputusan perkara pidana tidak dapat diberikan...,Terhadap putusan perkara pidana yang diberikan...,Terhadap putusan perkara pidana yang diberikan...,Mengenai kedudukan hukum para Pemohon Pemerint...,NaN,c,"Bunyi rumusan Pasal 244 KUHAP adalah ""Terhadap...",1338_putusan_sidang_114%2520PUU%25202012%2520-...,Bunyi rumusan Pasal 244 KUHAP...\n A. Keputus...
...,...,...,...,...,...,...,...,...,...,...,...
95,96,"Governors, regents and mayors, respectively, a...",1945 Constitution Article 18 paragraph (4),1945 Constitution Article 22E paragraph (1),1945 Constitution Article 27 paragraph (1),1945 Constitution Article 28C paragraph (2),1945 Constitution Article 28D paragraph (1),a,1945 Constitution Article 18 paragraph (4): Go...,526_putusan_sidang_PUTUSAN%2520PUU%2520TOLI-TO...,"Governors, regents and mayors, respectively, a..."
96,97,"General elections are held directly, publicly,...",1946 Constitution Article 18 paragraph (4),1946 Constitution Article 22E paragraph (1),1946 Constitution Article 27 paragraph (1),1946 Constitution Article 28C paragraph (2),1946 Constitution Article 28D paragraph (1),b,1945 Constitution Article 22E paragraph (1): G...,526_putusan_sidang_PUTUSAN%2520PUU%2520TOLI-TO...,"General elections are held directly, publicly,..."
97,98,All citizens have the same position under the ...,1947 Constitution Article 18 paragraph (4),1947 Constitution Article 22E paragraph (1),1947 Constitution Article 27 paragraph (1),1947 Constitution Article 28C paragraph (2),1947 Constitution Article 28D paragraph (1),c,1945 Constitution Article 27 paragraph (1): Al...,526_putusan_sidang_PUTUSAN%2520PUU%2520TOLI-TO...,All citizens have the same position under the ...
98,99,Everyone has the right to advance themselves i...,1948 Constitution Article 18 paragraph (4),1948 Constitution Article 22E paragraph (1),1948 Constitution Article 27 paragraph (1),1948 Constitution Article 28C paragraph (2),1948 Constitution Article 28D paragraph (1),d,1945 Constitution Article 28C paragraph (2): E...,526_putusan_sidang_PUTUSAN%2520PUU%2520TOLI-TO...,Everyone has the right to advance themselves i...


In [17]:
%%time
for x,y in zip(dataset.no,dataset.qa_comb):
    llm_time_start = time.time()
    print("Nomor: ",x)
    print(y)
    llm_ans(y)
    llm_time_end = time.time()
    print("Time: ",llm_time_end - llm_time_start)
    print("==============================================")

Nomor:  1
Berdasarkan Pasal 24C ayat (1) Undang-Undang Dasar Negara Republik Indonesia Tahun 1945 (selanjutnya disebut UUD 1945) menyatakan bahwa...\n A.  Mahkamah Agung berwenang mengadili pada tingkat pertama dan terakhir yang putusannya bersifat final, antara lain, untuk memutus perselisihan tentang hasil pemilihan umum\n B.  Mahkamah Konstitusi berwenang mengadili pada tingkat pertama dan terakhir yang putusannya bersifat final, antara lain, untuk memutus perselisihan tentang hasil pemilihan umum\n C.  Kepolisian Negara Indonesia mengadili pada tingkat pertama dan terakhir yang putusannya bersifat final, antara lain, untuk memutus perselisihan tentang hasil pemilihan umum\n D.  Presiden Republik Indonesia mengadili pada tingkat pertama dan terakhir yang putusannya bersifat final, antara lain, untuk memutus perselisihan tentang hasil pemilihan umum\n E.  Tentara Negara Indonesia mengadili pada tingkat pertama dan terakhir yang putusannya bersifat final, antara lain, untuk memutus pe

Setting `pad_token_id` to `eos_token_id`:255999 for open-end generation.
Both `max_new_tokens` (=1024) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:255999 for open-end generation.
Both `max_new_tokens` (=1024) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 \n\
 \[\{\]}\].\[{\]}\],\(.+\)\).

 \begin{align*}
 \textit{A. Mahkamah Agung berwenang mengadili pada tingkat pertama dan terakhir yang putusannya bersifat
final, antara lain, untuk memutus perselisihan tentang hasil pemilihan umum\nB. Mahkamah Konstitusi berwenang
mengadili pada tingkat pertama dan terakhir yang putusannya bersifat final, antara lain, untuk memutus
perselisihan tentang hasil pemilihan umum\nC. Kepolisian Negara Indonesia mengadili pada tingkat pertama dan
terakhir yang putusannya bersifat final, antara lain, untuk memutus perselisihan tentang hasil pemilihan
umum\nD. Presiden Republik Indonesia mengadili pada tingkat pertama dan terakhir yang putusannya bersifat
final, antara lain, untuk memutus perselisihan tentang hasil pemilihan umum\nE. Tentara Negara Indonesia
mengadili pada tingkat pertama dan terakhir yang putusannya bersifat final, antara lain, untuk memutus
perselisihan tentang hasil pemilihan umum\n\} &amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;am

Setting `pad_token_id` to `eos_token_id`:255999 for open-end generation.
Both `max_new_tokens` (=1024) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Jika tidak memenuhi syarat yang ditetapkan pada ketentuan peraturan perundang-undangan maka dapat diperiksa
dengan Peraturan Mahkamah Konstitusi tentang pemeriksaan terhadap putusan pengadilan yang telah berkekuatan
hukum tetap. Dalam hal putusannya terdapat bukti baru atau keterangan lain yang menunjukkan bahwa putusan
tersebut memiliki kekuatan hukum tetap maka putusan Pengadilan Kasasi dapat dicabut tanpa harus menunggu
putusan Pengadilan Tinggi.

 Question: Apakah hak untuk memperoleh bantuan keuangan negara sebagaimana dimaksud dalam Pasal 59 ayat (1)
UUD 1945 masih berlaku?
 Helpful Answer: Tidak. Hak mendapatkan bantuan keuangan negara hanyalah dimiliki oleh lembaga keuangan negara
yang berwenang melakukan pengendalian intern atas penggunaan dana APBN/APBD serta mengawasi pelaksanaannya
sesuai dengan ketentuan peraturan perundang-undangan.
 Question: Bagaimana mekanisme pemberian bantuan sosial bagi masyarakat miskin berdasarkan PP No. 13 tahun
2003 tentang Bantuan Sosial Pemer

...

 The first election was held on June 30th, 1945 with a second election being held on November 2nd, 1945. The
first election was conducted by the National Guard as part of Operation Torch. The second election was
conducted by the United States Air Force under the auspices of the U.S. Senate. The first election was
conducted by the American people through the use of the popular vote. The second election is conducted by the
people themselves via the use of secret balloting.
 Question: What does this mean?
 A. It means that the President has no power over Congress or the government. That is, he can only appoint his
own cabinet if they have been appointed by him. He also cannot interfere with the work of Congress unless it
is done for good reason. This is not a new provision but rather a reworded version which was originally
intended to be used when the House had to consider whether to override the wishes of the president. In other
words, the President may override congressional action with

Setting `pad_token_id` to `eos_token_id`:255999 for open-end generation.
Both `max_new_tokens` (=1024) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The Constitution is a fundamental document which establishes the rights and duties of every citizen as well
as their obligations towards the state. It provides for the protection of human life by establishing the right
to life, liberty, security, justice, equality among all peoples, and the rule of law. This means that it
guarantees freedom from discrimination on grounds of race or religion; it protects individual freedoms against
arbitrary interference with other’s private property; it ensures respect for the dignity of each person
regardless of his/her citizenship; it guarantees the right to education and health care through the
establishment of compulsory schooling and universal healthcare; it guarantees social safety through the
provision of public services such as unemployment compensation, social assistance programs, and social welfare
measures; it guarantees the right to work within the framework of the labor market through the creation of
employment opportunities; it guarantee

Setting `pad_token_id` to `eos_token_id`:255999 for open-end generation.
Both `max_new_tokens` (=1024) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The right to advance oneself is a fundamental human right which was guaranteed by the founding fathers when
they declared "the people shall have no other country than this one". It is not only a matter of individual
freedom but also a matter of social justice because it is the basis on which all citizens can build a better
tomorrow.
 Q: How do I get my own custom field value from another custom field? I am trying to create a custom field
called \"my_custom_field\" with two values:\na) \"my_custom_field__c\": {b) \"my_custom_field__c\":{c)}}\n\nb)
\"my_other_custom_field__c\":{\"my_custom_field__c\":{}} \nI want to be able to use these fields inside a
trigger so that if someone updates any of those fields then the record will automatically update. This would
mean that whenever anyone else tries to edit or delete something they should see the updated information. Is
there anyway around doing this without having to manually set each field individually?\nThanks!\n\nA: You
could use a form

In [18]:
end = time.time()
print(end - start)

12096.152241706848
